In [1]:
import random
import numpy as np
import copy 
import queue
import matplotlib.pyplot as plt
import pulp
import pandas as pd
import time 
import sympy as sp

class Building:
    counter = 0
    def __init__(self, num_resources=3, max_build_count=(1, 15), building_size_bounds=(1, 50), infinite_prob=0.0, infinite_buildings=False, struct_value_bounds=(1, 3)):
        self.name = f"B{Building.counter}"
        Building.counter += 1
        if infinite_buildings or np.random.rand() < infinite_prob:
            self.max_build_count = 99999
        else:
            self.max_build_count = np.random.randint(*max_build_count)
        self.building_size = np.random.randint(*building_size_bounds)
        self.built_locations = []
        self.build_rem = self.max_build_count
        bounds_avg = (building_size_bounds[1] + building_size_bounds[0]) / 2
        self.resource_values = np.round(np.random.uniform(struct_value_bounds[0]*self.building_size/bounds_avg, struct_value_bounds[1]*self.building_size/bounds_avg, num_resources), 2)
        self.effiency = self.resource_values/self.building_size

    def display(self):
        print()
        print(f"Name: {self.name}")
        print(f"Building Resource Values: {self.resource_values}")
        print(f"Max Build Count: {self.max_build_count}")
        print(f"Building Size: {self.building_size}")
        print(f"Built Locations: {self.built_locations}")
        print(f"Building Remaining: {self.build_rem}")
        print(f"Building Efficency: {np.round(self.effiency, 4)}")
        print(f"Total Building Efficency: {np.round(sum(self.effiency), 4)}")
        print()
    

class Land:
    counter = 0
    def __init__(self, space_range=(100, 500), num_resources=3, bounds=(0,3,.2), rounding=2, exp_rate=(0,3,.5), decay = (10,30,5)):
        self.name = f"L{Land.counter}"
        Land.counter += 1
        self.space = np.random.randint(space_range[0], space_range[1] + 1)
        self.resource_coef = np.round(np.clip(np.random.normal((bounds[1]+bounds[0])/2    , bounds[2]  , num_resources), bounds[0]  , bounds[1])  , rounding)
        self.resource_rate = np.round(np.clip(np.random.normal((exp_rate[1]+exp_rate[0])/2, exp_rate[2], num_resources), exp_rate[0], exp_rate[1]), rounding)
        self.decay         = np.round(np.clip(np.random.normal((decay[1]+decay[0])/2      , decay[2]   , num_resources), decay[0]   , decay[1])   , rounding)
        self.resource_vals = list(zip(self.resource_coef, self.resource_rate, self.decay))
        self.rem_space = self.space
        self.built_buildings = []
        self.building_prod = [0]*num_resources
        self.calc_prod = [0]*num_resources
        self.exp_growth_funcs = [self.exp_growth_func(coef, rate, decay) for coef, rate, decay in self.resource_vals]
        self.rate = exp_rate
        self.rounding = rounding
        self.num_resources = num_resources
        self.curr_eff_score = self.update_estimater()
        self.upper_bound = None
        #self.eff_score_short_term = self.calc_eff_short()
        #self.eff_score_long_term = self.calc_eff_long()
        self.lower_est = None
        self.upper_est = None
        self.mid_est = None
        self.potenial_est = None
    
    def calc_production(self, resource_idx, production):
        return self.exp_growth_funcs[resource_idx](production)

    def update_estimater(self):
        eff_scores = [0]*self.num_resources
        x = sp.symbols('x')
        for i in range(self.num_resources):
            func = self.resource_coef[i] * (1 + self.resource_rate[i])**(x/self.decay[i])
            x_val = self.building_prod[i]
            derivitive_x = func.subs(x, x_val)
            eff_scores[i] = float(derivitive_x)
        return eff_scores

    def update_prod(self):
        for i in range(self.num_resources):
            self.calc_prod[i] = self.exp_growth_funcs[i](self.building_prod[i])

    def test_prod(self, building):
        if self.construct_building(building):
            temp_prod = self.calc_prod
            self.remove_building(building)
            return temp_prod
        else:
            return self.calc_prod


    def exp_growth_func(self, coef, rate, decay):
        def resource_func(building_prod):
            return (coef * (1 + rate) ** (building_prod/decay)) - coef
        return resource_func

    def construct_building(self, building):
        # Run a check to see if the building can be made when looking at size/build contraints
        if building.build_rem > 0 and building.building_size <= self.rem_space:
            # Update Land variables
            self.rem_space -= building.building_size
            self.built_buildings.append(building.name)

            # Update Building variables
            building.build_rem -= 1
            building.built_locations.append(self.name)

            # Update resource production
            self.building_prod += building.resource_values
            self.update_prod()
            self.curr_eff_score = self.update_estimater()
            return True
        else:
            return False
            
    def remove_building(self, building):
        if building.name in self.built_buildings:
            # Update Land variables
            self.rem_space += building.building_size
            self.built_buildings.remove(building.name)

            # Update Building variables
            building.build_rem += 1
            building.built_locations.remove(self.name)

            # Update resource production
            self.building_prod -= building.resource_values
            self.update_prod()
            self.curr_eff_score = self.update_estimater()

    def display_resources(self):
        equations = f"\n"
        for i in range(self.num_resources):
            equations += f"Eq for R{i+1}: "
            equations += f"({self.resource_coef[i]} * (1 + {self.resource_rate[i]})^(x/{self.decay[i]}))) - {self.resource_coef[i]} \n"
        return equations

    def update_upper(self, val):
        self.upper_bound = np.array(val)
    
    def update_est(self):
        mid_point = self.upper_bound/2
        lower_point = mid_point/2
        upper_point = mid_point + lower_point
        lower_est = [0]*self.num_resources
        mid_est = [0]*self.num_resources
        upper_est = [0]*self.num_resources
        x = sp.symbols('x')
        for i in range(self.num_resources):
            func = self.resource_coef[i] * (1 + self.resource_rate[i])**(x/self.decay[i])
            lower_est[i] = float(func.subs(x, lower_point[i]))
            mid_est[i] = float(func.subs(x, mid_point[i]))
            upper_est[i] =float(func.subs(x, upper_point[i]))
        self.lower_est = lower_est
        self.upper_est = upper_est
        self.mid_est = mid_est
        self.update_potenial()
    
    def update_potenial(self):
        p = [0]*self.num_resources
        for i in range(self.num_resources):
            p[i] = self.exp_growth_funcs[i](self.upper_bound[i])
        self.potenial_est = p


    def display(self):
        print()
        print(f"Name: {self.name}")
        print(f"Space: {self.space}")
        print(f"Remaining_Space: {self.rem_space}")
        print(f"Built Buildings List: {self.built_buildings}")
        print(f"Resource Coefficients: {self.resource_coef}")
        print(f"Resource Rates: {self.resource_rate}")
        print(f"Resource Decay: {self.decay}")
        print(f"Resource Equation: {self.display_resources()}")
        print(f"Building Production: {self.building_prod}")
        print(f"Calc Production: {np.round(self.calc_prod, 2)}")
        print(f"Curr Estimater: {np.round(self.curr_eff_score, 2)}")
        if self.upper_bound is not None:
            print(f"Upper Bound: {np.round(self.upper_bound, 2)}")
        if self.lower_est is not None:
            print(f"Lower Estimate: {np.round(self.lower_est, 2)}")
            print(f"Middle Estimate: {np.round(self.mid_est, 2)}")
            print(f"Upper Estimate: {np.round(self.upper_est, 2)}")
        print()

## Testing

In [2]:
x = Land()
b = Building()

In [3]:
x.display()
b.display()


Name: L0
Space: 316
Remaining_Space: 316
Built Buildings List: []
Resource Coefficients: [1.47 1.64 1.57]
Resource Rates: [0.94 1.4  2.27]
Resource Decay: [11.36 20.3  24.71]
Resource Equation: 
Eq for R1: (1.47 * (1 + 0.94)^(x/11.36))) - 1.47 
Eq for R2: (1.64 * (1 + 1.4)^(x/20.3))) - 1.64 
Eq for R3: (1.57 * (1 + 2.27)^(x/24.71))) - 1.57 

Building Production: [0, 0, 0]
Calc Production: [0 0 0]
Curr Estimater: [1.47 1.64 1.57]


Name: B0
Building Resource Values: [1.34 2.16 1.12]
Max Build Count: 3
Building Size: 20
Built Locations: []
Building Remaining: 3
Building Efficency: [0.067 0.108 0.056]
Total Building Efficency: 0.231



In [4]:
x.construct_building(b)

True

In [5]:
x.display()


Name: L0
Space: 316
Remaining_Space: 296
Built Buildings List: ['B0']
Resource Coefficients: [1.47 1.64 1.57]
Resource Rates: [0.94 1.4  2.27]
Resource Decay: [11.36 20.3  24.71]
Resource Equation: 
Eq for R1: (1.47 * (1 + 0.94)^(x/11.36))) - 1.47 
Eq for R2: (1.64 * (1 + 1.4)^(x/20.3))) - 1.64 
Eq for R3: (1.57 * (1 + 2.27)^(x/24.71))) - 1.57 

Building Production: [1.34 2.16 1.12]
Calc Production: [0.12 0.16 0.09]
Curr Estimater: [1.59 1.8  1.66]



# Algorithms

## Helping Funcs

In [6]:
def update_lands(buildings, lands, num_resources):
    best_ratios = [0]*num_resources
    for b in buildings:
        eff = b.effiency
        best_ratios = [max(z,y) for z,y in zip(best_ratios, eff)]
    for l in lands:
        upper_bounds = [l.space * x for x in best_ratios]
        l.update_upper(upper_bounds)
        l.update_est()

def sparsity_score(max_build_count, building_size_bounds, space_range, num_buildings, num_land):
    avg_land_space_range = (space_range[0] + space_range[1]) / 2.0
    avg_building_size = (building_size_bounds[0] + building_size_bounds[1]) / 2.0
    avg_building_count = (max_build_count[0] + max_build_count[1]) / 2.0
    sparsity_ratio = (avg_building_count * avg_building_size * num_buildings) / (avg_land_space_range * num_land)
    return sparsity_ratio

def calc_prod(lands, num_resources):
    total_production = np.zeros(num_resources)
    for land in lands:
        total_production += land.calc_prod
    return total_production

## Random

In [7]:
# Random Algorithm to distribute buildings randomly among lands
# Algorithm Summary: Choose a piece of Land -> Fill Land with random buildings -> When full move to next land and repeat
def random_algorithm(buildings, lands):
    lands_copy = copy.deepcopy(lands)
    buildings_copy = copy.deepcopy(buildings)
    for land in lands_copy:
        while land.rem_space > 0:
            # Filter available buildings that can fit in the remaining space
            available_buildings = [b for b in buildings_copy if b.building_size <= land.rem_space and b.build_rem > 0]
            # No available buildings that can fit
            if not available_buildings:
                break
            building = random.choice(available_buildings)
            land.construct_building(building)
    return buildings_copy, lands_copy

## Greedy

In [8]:
# Greedy Algorithm to greedyly choose best building per land
# Algorithm Summary: Create queue of best land based on resource values, then create queue based on best building/space efficiency,
# place buildings until queue empty or land is filled repeat for all land
def greedy_algorithm(buildings, lands):
    lands_copy = copy.deepcopy(lands)
    buildings_copy = copy.deepcopy(buildings)

    # Sort lands by total resource values, richest land is at the top of the queue
    lands_copy.sort(key=lambda land: sum(land.curr_eff_score), reverse=True)

    for land in lands_copy:
        # Sort buildings by space efficiency (resource values*land resources)/building size
        buildings_copy.sort(key=lambda building: sum((land.curr_eff_score * building.effiency)), reverse=True)

        # Add most efficent building until no longer possible
        for building in buildings_copy:
            while (land.rem_space >= building.building_size) and building.build_rem >= 1:
                land.construct_building(building)

    return buildings_copy, lands_copy

## LP + Helper Methods

In [9]:
def lp_algorithm(buildings, lands, estimater = "u", timelimit = 1):
    lands_copy = copy.deepcopy(lands)
    buildings_copy = copy.deepcopy(buildings)
    model = pulp.LpProblem("Building_Allocation", pulp.LpMaximize)
    # Create a dictionary to hold decision variables
    building_vars = {}
    for building in buildings:
        for land in lands:
            building_vars[(building.name, land.name)] = pulp.LpVariable(
        f"B_{building.name}_on_L{land.name}", lowBound=0, upBound=building.max_build_count, cat='Integer'
    )
            
    objective = []
    for land in lands:
        if estimater == "u":
            land_richness = land.upper_est
        if estimater == "l":
            land_richness = land.lower_est
        if estimater == "m":
            land_richness = land.mid_est
        for building in buildings:
            building_production = building.resource_values
            for i in range(len(land_richness)):
                objective.append(
                    building_vars[(building.name, land.name)] * land_richness[i] * building_production[i]
                )

    model += pulp.lpSum(objective), "Objective"
    for land in lands:
        land_space = land.space
        space_constraint = pulp.lpSum(
            building_vars[(building.name, land.name)] * building.building_size for building in buildings
        )
        model += space_constraint <= land_space, f"Land_{land.name}_Space_Constraint"
        
    for building in buildings:
        build_count = building.build_rem
        usage_constraint = pulp.lpSum(building_vars[(building.name, land.name)] for land in lands)
        model += usage_constraint <= build_count, f"Building_{building.name}_Usage_Constraint"

    model.solve(pulp.PULP_CBC_CMD(msg=False, timeLimit=timelimit))

    for (building_name, land_name), var in building_vars.items():
        if var.varValue > 0:
            for b in buildings_copy:
                for l in lands_copy:
                    if b.name == building_name and l.name == land_name:
                        for i in range(int(var.varValue)):
                            l.construct_building(b)
            #print(f"Allocate {var.varValue} times Building {building_name} on Land {land_name}")
    return buildings_copy, lands_copy

def calculate_LP_production(buildings, lands, building_vars):
    num_resources = lands[0].num_resources
    production_scores = [0.0] * num_resources

    for land in lands:
        for building in buildings:
            building_production = building.resource_values
            for i in range(num_resources):
                production_scores[i] += (
                    building_vars[(building.name, land.name)].varValue
                    * land.calc_production(i, building_production[i])
                )

    return [round(score, 2) for score in production_scores]

## Random Swap

In [10]:
def random_swap(buildings, lands, num_resources, max_iterations=1000):
    best_buildings = copy.deepcopy(buildings)
    best_lands = copy.deepcopy(lands)
    best_score = calc_prod(lands, num_resources)
    scores = []
    for i in range(max_iterations):
        # Randomly select two buildings
        building1 = random.choice(best_buildings)
        building2 = random.choice(best_buildings)

        # Randomly select two lands
        land1 = random.choice(best_lands)
        land2 = random.choice(best_lands)

        # See if they can be swapped
        if is_feasible_swap(land1, land2, building1, building2):
            land1.remove_building(building1)
            land1.construct_building(building2)

            land2.remove_building(building2)
            land2.construct_building(building1)

            new_score = calc_prod(best_lands, num_resources)

            # If the new score is better, keep the swap
            if sum(new_score) > sum(best_score):
                best_score = new_score
            else:
                # Revert the swap if the new score is not better
                land1.remove_building(building2)
                land1.construct_building(building1)
                land2.remove_building(building1)
                land2.construct_building(building2)
    return best_score

def is_feasible_swap(land1, land2, building1, building2):
    return (land1.rem_space + building1.building_size >= building2.building_size and
            land2.rem_space + building2.building_size >= building1.building_size)


def calc_prod(lands, num_resources):
    total_production = np.zeros(num_resources)
    for land in lands:
        total_production += land.calc_prod
    return total_production


## Rule-Based Using Estimators + Heuristics 

In [11]:
def rule_based(buildings, lands):
    buildings_copy = copy.deepcopy(buildings)
    lands_copy = copy.deepcopy(lands)

    lands_copy.sort(key=lambda land: sum(land.potenial_est), reverse=True)
    for land in lands_copy:
        buildings_copy.sort(key=lambda building: sum((land.potenial_est * building.effiency)), reverse=True)
        for building in buildings_copy:
            while (land.rem_space >= building.building_size) and building.build_rem >= 1:
                land.construct_building(building)
    return buildings_copy, lands_copy

# Testing Space

In [12]:
############################### Problem Space Variables ################################

# Resources Params
num_resources=3

# Problem size 
num_buildings = 20
num_land = 7

############## GEN SPACE ######################
buildings = [Building() for _ in range(num_buildings)]
lands = [Land() for _ in range(num_land)]

In [13]:
update_lands(buildings, lands, num_resources)

buildings_ran, lands_ran = random_algorithm(buildings, lands)
rand_prod = calc_prod(lands_ran, num_resources)
print(rand_prod)
print(sum(rand_prod))

buildings_greedy, lands_greedy = greedy_algorithm(buildings, lands)
buildings_prod = calc_prod(lands_greedy, num_resources)
print(buildings_prod)
print(sum(buildings_prod))

buildings_lp_upper, lands_lp_upper = lp_algorithm(buildings, lands, "u")
lp_upper_prod = calc_prod(lands_lp_upper, num_resources)
print(lp_upper_prod)
print(sum(lp_upper_prod))

buildings_lp_lower, lands_lp_lower = lp_algorithm(buildings, lands, "l")
lp_lower_prod = calc_prod(lands_lp_lower, num_resources)
print(lp_lower_prod)
print(sum(lp_lower_prod))

buildings_lp_middle, lands_lp_middle = lp_algorithm(buildings, lands, "m")
lp_middle_prod = calc_prod(lands_lp_middle, num_resources)
print(lp_middle_prod)
print(sum(lp_middle_prod))

ran_swap_prod = random_swap(buildings_greedy, lands_greedy, num_resources)
print(ran_swap_prod)
print(sum(ran_swap_prod))

buildings_rule, lands_rule = rule_based(buildings, lands)
rule_prod = calc_prod(lands_rule, num_resources)
print(rule_prod)
print(sum(rule_prod))

[29.88727147 40.55936464 23.05548876]
93.50212486997403
[21.57416057 56.34482201 27.61501878]
105.53400135033243
[57.49426694 86.13441427 28.08588296]
171.71456416654624
[55.3411365  76.94228351 27.37388238]
159.65730239389706
[57.5214622  86.04770278 28.08251009]
171.65167506445658
[21.57416057 56.34482201 27.61501878]
105.53400135033243
[57.43396737 83.5994824  27.30135031]
168.3348000777761


In [14]:
small = [8, 5, 2, "Small"]
normal_ = [20, 10, 3, "Normal"]
large = [35, 15, 5, "Large"]

v_sparse = [(3, 5), (5, 10)]
sparse = [(3, 8), (5, 20)]
normal = [(5, 15), (10, 30)]
rich = [(10, 20), (20, 40)]
v_rich = [(1, 5), (9999, 99999)]

seeds = [i+100 for i in range(10)]

In [15]:
scores_df = pd.DataFrame(columns=["Greedy", "Random", "Lp_Upper", "Lp_Middle", "Lp_Lower", "Rand+Swap", "Heuristics + RB", "Sparsity_Score", "Problem_Space"])
runtime_df = pd.DataFrame(columns=["Greedy", "Random", "Lp_Upper", "Lp_Middle", "Lp_Lower", "Rand+Swap", "Heuristics + RB", "Problem_Space"])

In [16]:
for space in [small, normal_, large]:
    for b in [v_sparse, sparse, normal, rich, v_rich]:
        greedy_score_n = 0
        random_score_n = 0
        lp_upper_score_n = 0
        lp_middle_score_n = 0
        lp_lower_score_n = 0
        rand_swap_score_n = 0
        rule_based_score_n = 0

        greedy_time = 0
        random_time = 0
        lp_upper_time = 0
        lp_middle_time = 0
        lp_lower_time = 0
        rand_swap_time = 0
        rule_based_time = 0
        print("next")
        for s in seeds:
            greedy_score = 0
            random_score = 0
            lp_upper_score = 0
            lp_middle_score = 0
            lp_lower_score = 0
            rand_swap_score = 0
            rule_based_score = 0

            num_buildings = space[0]
            num_land = space[1]
            num_resources = space[2]
            building_size_bounds = b[0]
            max_build_count = b[1]
            #print("===================================")
            #score = sparsity_score(max_build_count, building_size_bounds, space_range, num_buildings, num_land)
            #print(f"Sparsity Score: {score}")
            random_seed = s
            random.seed(random_seed)
            np.random.seed(random_seed)
            buildings = [Building(max_build_count=max_build_count, num_resources=num_resources, building_size_bounds=building_size_bounds) for _ in range(num_buildings)]
            lands = [Land(num_resources=num_resources) for _ in range(num_land)]

            # Calcs some estimaters/heuristics which are needed for some of the algos
            update_lands(buildings, lands, num_resources)

            # Calculate and display total production score

            start_time = time.time()
            buildings_ran, lands_ran = random_algorithm(buildings, lands)
            random_score = sum(calc_prod(lands_ran, num_resources))
            #print(f"Total Production Score with Random Algorithm: {total_production_score} \nTotal Production: {round(sum(total_production_score), 2)}")
            end_time = time.time()
            random_time += end_time - start_time

            start_time = time.time()
            greedy_buildings, greedy_land = greedy_algorithm(buildings, lands)
            greedy_score = sum(calc_prod(greedy_land, num_resources))
            #print(f"Total Production Score with Greedy Algorithm: {total_production_score} \nTotal Production: {round(sum(total_production_score), 2)}")
            end_time = time.time()
            greedy_time += end_time - start_time

            start_time = time.time()
            buildings_lp_upper, lands_lp_upper = lp_algorithm(buildings, lands, "u")
            lp_upper_score = sum(calc_prod(lands_lp_upper, num_resources))
            #print(f"Total Production Score with LP Algorithm: {total_production_score} \nTotal Production: {round(sum(total_production_score), 2)}")
            end_time = time.time()
            lp_upper_time += end_time - start_time

            start_time = time.time()
            buildings_lp_lower, lands_lp_lower = lp_algorithm(buildings, lands, "l")
            lp_lower_score = sum(calc_prod(lands_lp_lower, num_resources))
            #print(f"Total Production Score with LP Algorithm: {total_production_score} \nTotal Production: {round(sum(total_production_score), 2)}")
            end_time = time.time()
            lp_lower_time += end_time - start_time

            start_time = time.time()
            buildings_lp_middle, lands_lp_middle = lp_algorithm(buildings, lands, "m")
            lp_middle_score = sum(calc_prod(lands_lp_middle, num_resources))
            #print(f"Total Production Score with LP Algorithm: {total_production_score} \nTotal Production: {round(sum(total_production_score), 2)}")
            end_time = time.time()
            lp_middle_time += end_time - start_time

            start_time = time.time()
            rand_swap_score = sum(random_swap(buildings_ran, lands_ran, num_resources))
            end_time = time.time()
            rand_swap_time += end_time - start_time

            start_time = time.time()
            buildings_rule, lands_rule = rule_based(buildings, lands)
            rule_based_score = sum(calc_prod(lands_rule, num_resources))
            #print(f"Total Production Score with Hill Climbing Algorithm: {best_score} \nTotal Production: {round(sum(best_score), 2)}")
            end_time = time.time()
            rule_based_time += end_time - start_time

            best_score = max(greedy_score, random_score, lp_upper_score, lp_middle_score, lp_lower_score, rand_swap_score, rule_based_score)
            greedy_score, random_score, lp_upper_score, lp_middle_score, lp_lower_score, rand_swap_score, rule_based_score = \
            greedy_score/best_score, random_score/best_score, lp_upper_score/best_score, lp_middle_score/best_score, lp_lower_score/best_score, rand_swap_score/best_score, rule_based_score/best_score

            greedy_score_n += greedy_score
            random_score_n += random_score
            lp_upper_score_n += lp_upper_score
            lp_middle_score_n += lp_middle_score
            lp_lower_score_n += lp_lower_score
            rand_swap_score_n += rand_swap_score
            rule_based_score_n += rule_based_score

        score = sparsity_score(max_build_count, building_size_bounds, (100, 500), num_buildings, num_land)
        if score > 10:
            score = "inf"
        else:
            score = round(score,2)
        scores_df.loc[len(scores_df)] = [round(greedy_score_n, 6), round(random_score_n, 6), round(lp_upper_score_n, 6), round(lp_middle_score_n, 6), round(lp_lower_score_n, 6),\
                                          round(rand_swap_score_n, 6), round(rule_based_score_n, 6), score, space[3]]
        runtime_df.loc[len(runtime_df)] = [greedy_time, random_time, lp_upper_time, lp_middle_time, lp_lower_time, rand_swap_time, rule_based_time, space[3]]

next
next
next
next
next
next
next
next
next
next
next
next
next
next
next


In [17]:
scores_df

,Greedy,Random,Lp_Upper,Lp_Middle,Lp_Lower,Rand+Swap,Heuristics + RB,Sparsity_Score,Problem_Space
0,6.556779,6.171403,8.135939,8.135939,7.845644,6.171403,9.423180,0.16,Small
1,4.229318,1.738050,9.997881,9.711481,9.354943,1.738050,9.993684,0.37,Small
2,4.991952,3.574627,9.995149,9.776539,9.387988,3.682678,9.731081,1.07,Small
3,7.514101,4.617150,9.981034,9.920332,9.789298,5.864454,9.582273,2.4,Small
4,7.101022,0.101626,9.107690,9.103736,8.474167,0.219470,9.993718,inf,Small
5,0.788580,0.168909,9.946284,9.790505,8.006764,0.168909,9.935645,0.2,Normal
6,0.303887,0.151369,9.994612,9.832001,8.527119,0.154637,9.566356,0.46,Normal
7,3.076467,1.200491,9.816402,9.663912,9.358858,1.292899,9.849290,1.33,Normal
8,5.437325,2.485400,10.000000,9.767957,9.481496,2.931137,9.501711,3.0,Normal
9,3.864379,0.002724,8.139301,8.139299,8.476418,0.005508,9.498681,inf,Normal


In [18]:
runtime_df

,Greedy,Random,Lp_Upper,Lp_Middle,Lp_Lower,Rand+Swap,Heuristics + RB,Problem_Space
0,0.491767,0.503060,0.720850,0.349951,0.404669,34.803865,0.491145,Small
1,0.921674,0.877164,1.093238,0.720645,0.824116,32.880193,0.914085,Small
2,1.315992,1.332875,1.837681,1.615690,2.667949,27.194824,1.315658,Small
3,0.885876,0.945510,4.049159,3.304897,5.053589,18.574244,0.881885,Small
4,8.608121,5.256253,8.568394,8.781800,8.198682,12.096942,7.978210,Small
5,1.909862,1.960834,2.303824,1.794766,1.910017,48.838482,1.902593,Normal
6,3.509260,3.491921,4.173868,4.021633,3.771367,56.954333,3.343337,Normal
7,4.193743,4.195989,12.069063,13.910463,14.105266,45.299096,4.134215,Normal
8,2.709559,2.906987,9.518329,12.438791,12.607790,31.673755,2.742501,Normal
9,20.866480,17.693804,22.190810,21.744771,20.274138,17.319052,22.275126,Normal


In [19]:
runtime_df.to_csv(path_or_buf="runtime.csv", index = False)

In [20]:
scores_df.to_csv(path_or_buf="scores.csv", index = False)

In [22]:
averages = scores_df["Heuristics + RB"].mean()
averages

9.739724133333334

In [23]:
averages = scores_df["Lp_Upper"].mean()
averages

9.594569533333331

In [24]:
averages = scores_df["Lp_Middle"].mean()
averages

9.393049666666666

In [25]:
averages = scores_df["Lp_Lower"].mean()
averages

8.669131066666665